In [1]:
import requests
import json 
import pandas as pd
import pymongo
import os

In [2]:
# Calling api and converting the data into readable form

url = f'https://archive-api.open-meteo.com/v1/archive?latitude=16.0678&longitude=108.2208&start_date=1993-09-30&end_date=2023-09-30&daily=weathercode,temperature_2m_mean,precipitation_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,et0_fao_evapotranspiration&timezone=auto'
response = requests.get(url)
data = json.loads(response.text)

In [ ]:
# Use json_normalize to turn data into pandas dataframe

df = pd.json_normalize(data)

# Use pandas methods to transform the dataframe

df = df.apply(pd.Series.explode)
df = df.reset_index(drop = True)
df.head()

In [ ]:
# Modify columns name

df.columns = df.columns.str.replace('daily.', 'data-')
df.columns = df.columns.str.replace('daily_units.', 'unit-')
df.head()

In [ ]:
df.columns

In [6]:
# If want to save to MongoDB database then change this value into True

installedMongo = False
if installedMongo:
    # Connect to localhost database
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client['weather']
    collection = db['weather-daily']
    
    # Convert dataframe
    insertData = df.to_dict(orient="records")   
    
    # Insert into MongoDB collection
    inserted_ids = collection.insert_many(insertData).inserted_ids

    client.close()
else:
    if not os.path.isdir('../data'):
        os.makedirs('../data')

    df.to_csv('../data/dailyWeather.csv', index=False)